In [1]:
import pickle
import os
import pandas as pd
import numpy as np
import tqdm

import torch
import torch.nn as nn
import torchvision.models as models


### Averaging predictions

In [9]:
path_to_data = '/data/rustem/landmarks/data'
path_to_submit = '/data/rustem/landmarks/data/checkpoints/resnext50_32x4d_06_folds/submit0_5.csv'
fold_index = 6
test_predictions_list = []
for i in range(6):
    path_to_predictions = f'/data/rustem/landmarks/data/checkpoints/resnext50_32x4d_0{fold_index}_fold{i}/resnext50_32x4d_test_predictions.pkl'
    test_predictions_list.append(pickle.load(open(path_to_predictions, 'rb')))

In [10]:
SUBMISSION_HEADER = "file_name,Point_M0_X,Point_M0_Y,Point_M1_X,Point_M1_Y,Point_M2_X,Point_M2_Y,Point_M3_X,Point_M3_Y,Point_M4_X,Point_M4_Y,Point_M5_X,Point_M5_Y,Point_M6_X,Point_M6_Y,Point_M7_X,Point_M7_Y,Point_M8_X,Point_M8_Y,Point_M9_X,Point_M9_Y,Point_M10_X,Point_M10_Y,Point_M11_X,Point_M11_Y,Point_M12_X,Point_M12_Y,Point_M13_X,Point_M13_Y,Point_M14_X,Point_M14_Y,Point_M15_X,Point_M15_Y,Point_M16_X,Point_M16_Y,Point_M17_X,Point_M17_Y,Point_M18_X,Point_M18_Y,Point_M19_X,Point_M19_Y,Point_M20_X,Point_M20_Y,Point_M21_X,Point_M21_Y,Point_M22_X,Point_M22_Y,Point_M23_X,Point_M23_Y,Point_M24_X,Point_M24_Y,Point_M25_X,Point_M25_Y,Point_M26_X,Point_M26_Y,Point_M27_X,Point_M27_Y,Point_M28_X,Point_M28_Y,Point_M29_X,Point_M29_Y\n"
def create_submission(path_to_data, test_predictions_list, path_to_submission_file):
    test_dir = os.path.join(path_to_data, "test")

    output_file = path_to_submission_file
    wf = open(output_file, 'w')
    wf.write(SUBMISSION_HEADER)

    mapping_path = os.path.join(test_dir, 'test_points.csv')
    mapping = pd.read_csv(mapping_path, delimiter='\t')

    for i, row in tqdm.tqdm(mapping.iterrows()):
        file_name = row[0]
        point_index_list = np.array(eval(row[1]))
        needed_points = []
        for test_predictions in test_predictions_list:
            points_for_image = test_predictions['landmarks'][i]
            needed_points.append(points_for_image[point_index_list])
        needed_points = np.stack(needed_points, axis=0)
        needed_points = np.median(needed_points, axis=0).astype(np.int)
            
        wf.write(file_name + ',' + ','.join(map(str, needed_points.reshape(2 * len(point_index_list)))) + '\n')

In [11]:
create_submission(path_to_data, test_predictions_list, path_to_submit)

99819it [00:44, 2257.79it/s]


### Averaging model weights

In [2]:
NUM_PTS = 971
model = models.resnext50_32x4d(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2 * NUM_PTS, bias=True)
model = nn.DataParallel(model)

In [3]:
path_to_data = '/data/rustem/landmarks/data'
model_weights = []
for i in range(5):
    path_to_predictions = f'/data/rustem/landmarks/data/checkpoints/resnext50_32x4d_00_fold{i}/resnext50_32x4d_best.pth'
    model_weights.append(torch.load(path_to_predictions, map_location='cpu'))

In [4]:
final_weights = model_weights[0].copy()
for param_name in tqdm.tqdm(model_weights[0]):
    all_weights = torch.stack([model_weights[i][param_name] for i in range(5)])
    if all_weights.dtype == torch.int64:
        mean_weight = torch.mean(all_weights.float(), dim=0).long()
    else:
        mean_weight = torch.mean(all_weights, dim=0)
    final_weights[param_name] = mean_weight

100%|██████████| 320/320 [00:56<00:00,  5.71it/s]


In [5]:
model.load_state_dict(final_weights)

<All keys matched successfully>

In [10]:
path_to_save_model_weights = '/data/rustem/landmarks/data/checkpoints/weight_average/weight_average.pth'
with open(path_to_save_model_weights, 'wb') as fp:
    torch.save(model.state_dict(), fp)